# Scraping Industries Data

In [ ]:
#take city data for cLat/cLon here: https://simplemaps.com/data/us-cities
import math
import pandas as pd

earth_radius = 3960.0
degrees_to_radians = math.pi/180.0
radians_to_degrees = 180.0/math.pi
#asdasd

#Importing
industries = pd.read_csv("/content/drive/MyDrive/pfas_python_files/preprocessing/industries.csv", encoding = 'unicode_escape', engine ='python')
uscities = pd.read_csv("/content/drive/MyDrive/pfas_python_files/preprocessing/uscities.csv", encoding = 'unicode_escape', engine ='python')


#Cleaning Data
industries = industries[(industries['Latitude'] != '-') & (industries['Longitude'] != '-')]
industries['Latitude'] = pd.to_numeric(industries['Latitude'], errors='coerce')
industries['Longitude'] = pd.to_numeric(industries['Longitude'], errors='coerce')
industries = industries[['Latitude', 'Longitude']]

uscities = uscities[['city', 'state_name', 'lat', 'lng', 'population', 'density', 'ranking']]
uscities['Latitude'] = pd.to_numeric(uscities['lat'], errors='coerce')
uscities['Longitude'] = pd.to_numeric(uscities['lng'], errors='coerce')
uscities = uscities.drop(['lat', 'lng'], axis=1)


#Global Variables
upper_boundary_miles = 0
lower_boundary_miles = 0
right_boundary_miles = 0
left_boundary_miles = 0

grid_radius = 2.5


# 2.5 miles = around 0.04578754578 longitude
# 2.5 miles = around 0.03623188405 latitude


def calculate_industry_density_in_each_city():

    uscities['Industry Count'] = 0
    for index, row in uscities.iterrows():
        current_city_longitude = row['Longitude']
        current_city_latitude = row['Latitude']

        # Horizontal Boundaries
        right_boundary_longitude = current_city_longitude + 0.04578754578
        left_boundary_longitude = current_city_longitude - 0.04578754578

        # Vertical Boundaries
        upper_boundary_latitude = current_city_latitude + 0.03623188405
        lower_boundary_latitude = current_city_latitude - 0.03623188405

        industry_count = 0

        for ind_index, ind_row in industries.iterrows():
            current_industry_longitude = ind_row['Longitude']
            current_industry_latitude = ind_row['Latitude']

            # Check each industry point to see if it falls within the 2.5 mile radius of each city
            if (left_boundary_longitude < current_industry_longitude < right_boundary_longitude) and \
               (lower_boundary_latitude < current_industry_latitude < upper_boundary_latitude):
                industry_count += 1




        uscities.at[index, 'Industry Count'] = industry_count

        print("Index:", index)
        print("City:", row["city"])
        print("Industry count:", industry_count)
        print("-------------------------------------")


calculate_industry_density_in_each_city()

columns_to_keep = ['city', 'state_name', 'Latitude', 'Longitude', 'Industry Count']

uscities = uscities[columns_to_keep]

uscities.to_csv('/content/drive/MyDrive/pfas_python_files/preprocessing/PFAS_Industries_Data.csv')

# Scraping EJScreen Data

In [ ]:
#change min/max values and run cell to complete assignments:
#https://docs.google.com/document/d/1MVm8UH17-1ERt5JKMV1ZecDkKyYpWPnqDKAWwcvlzfo/edit?usp=sharing

min = 0
max = 2

import math
import requests
import pandas as pd
import numpy as np


earth_radius = 3960.0
degrees_to_radians = math.pi/180.0
radians_to_degrees = 180.0/math.pi

uscities = pd.read_csv("/preprocessing/uscities.csv", encoding = 'unicode_escape', engine ='python')
uscities = uscities[['city','state_name','lat','lng','population','density','ranking']]
uscities = uscities.rename(columns={'lat': 'Latitude', 'lng': 'Longitude'})

def change_in_latitude(miles):
	return (miles/earth_radius)*radians_to_degrees

def change_in_longitude(latitude, miles):
	r = earth_radius*math.cos(latitude*degrees_to_radians)
	return (miles/r)*radians_to_degrees

#EJ SCREEN

def collect_ej_data(cLat, cLon, rad):
    BASE_URL = "https://geopub.epa.gov/arcgis/rest/services/ejscreen/ejscreen_v2023_with_as_cnmi_gu_vi/MapServer/exts/EJCensusReports/GetEJScreen"
    data = []
    d = requests.get(BASE_URL, params={"namestr" : "",
                                       "geometry" : '{"x": ' + str(cLon) + ', "y": ' + str(cLat) + ', "spatialReference" : {"wkid" : 4326}}',
                                       "distance" : rad, "unit" : 9035,
                                       "areatype" : "", "areaid" : "", "f" : "pjson"})
    data.append(d.json())
    df = pd.DataFrame(data)
    return df

def clean_df(ej_df, cols, str_cols):
    ej_df = ej_df[cols].copy()
    for col in cols:
        ej_df[col] = ej_df[col].map(lambda x: str(x).replace('%',''))
        ej_df[col] = ej_df[col].map(lambda x: str(x).replace('N/A',''))
        if col not in str_cols:
            ej_df[col] = pd.to_numeric(ej_df[col])
    return(ej_df)


ej_columns = ["RAW_E_PM25", "RAW_E_O3", "RAW_E_DIESEL", "RAW_E_CANCER", "RAW_E_RESP", "RAW_E_TRAFFIC", "RAW_E_LEAD",
              "RAW_E_NPL", "RAW_E_RMP", "RAW_E_TSDF", "RAW_E_UST", "RAW_E_NPDES", "RAW_D_INDEX", "RAW_D_MINOR",
              "RAW_D_INCOME", "RAW_D_UNEMPLOYED", "RAW_D_LING", "RAW_D_LESSHS", "RAW_D_UNDER5", "RAW_D_OVER64",
              "S_E_PM25", "S_E_O3", "S_E_DIESEL", "S_E_CANCER", "S_E_RESP", "S_E_TRAFFIC", "S_E_LEAD", "S_E_NPL",
              "S_E_RMP", "S_E_TSDF", "S_E_UST", "S_E_NPDES", "S_D_INDEX", "S_D_MINOR", "S_D_INCOME", "S_D_UNEMPLOYED",
              "S_D_LING", "S_D_LESSHS", "S_D_UNDER5", "S_D_OVER64", "S_P_PM25", "S_P_O3", "S_P_DIESEL", "S_P_CANCER",
              "S_P_RESP", "S_P_TRAFFIC", "S_P_LEAD", "S_P_NPL", "S_P_RMP", "S_P_TSDF", "S_P_UST", "S_P_NPDES",
              "S_E_PM25_PER", "S_E_O3_PER", "S_E_DIESEL_PER", "S_E_CANCER_PER", "S_E_RESP_PER", "S_E_TRAFFIC_PER",
              "S_E_LEAD_PER", "S_E_NPL_PER", "S_E_RMP_PER", "S_E_TSDF_PER", "S_E_UST_PER", "S_E_NPDES_PER", "S_D_INDEX_PER",
              "S_D_MINOR_PER", "S_D_INCOME_PER", "S_D_UNEMPLOYED_PER", "S_D_LING_PER", "S_D_LESSHS_PER", "S_D_UNDER5_PER",
              "S_D_OVER64_PER", "N_E_PM25", "N_E_O3", "N_E_DIESEL", "N_E_CANCER", "N_E_RESP", "N_E_TRAFFIC", "N_E_LEAD",
              "N_E_NPL", "N_E_RMP", "N_E_TSDF", "N_E_UST", "N_E_NPDES", "N_D_INDEX", "N_D_MINOR", "N_D_INCOME",
              "N_D_UNEMPLOYED", "N_D_LING", "N_D_LESSHS", "N_D_UNDER5", "N_D_OVER64", "N_P_PM25", "N_P_O3", "N_P_DIESEL",
              "N_P_CANCER", "N_P_RESP", "N_P_TRAFFIC", "N_P_LEAD", "N_P_NPL", "N_P_RMP", "N_P_TSDF",
              "N_P_UST", "N_P_NPDES", "N_E_PM25_PER", "N_E_O3_PER", "N_E_DIESEL_PER", "N_E_CANCER_PER",
              "N_E_RESP_PER", "N_E_TRAFFIC_PER", "N_E_LEAD_PER", "N_E_NPL_PER", "N_E_RMP_PER", "N_E_TSDF_PER",
              "N_E_UST_PER", "N_E_NPDES_PER", "N_D_INDEX_PER", "N_D_MINOR_PER", "N_D_INCOME_PER",
              "N_D_UNEMPLOYED_PER", "N_D_LING_PER", "N_D_LESSHS_PER", "N_D_UNDER5_PER", "N_D_OVER64_PER",
              "stateAbbr", "stateName", "epaRegion", "totalPop", "NUM_NPL", "NUM_TSDF"]

string_cols = ["stateAbbr", "stateName", "epaRegion", "totalPop", "NUM_NPL", "NUM_TSDF"]


def scrape(i):
    ej_scrape_df = collect_ej_data(uscities.iloc[i]['Latitude'], uscities.iloc[i]['Longitude'], 2.8217)
    clean_ej_scrape_df = clean_df(ej_scrape_df, ej_columns, string_cols)
    return clean_ej_scrape_df

final = pd.DataFrame()

i = min
while i < max:
    try:
        print("Current Iteration:", i)
        scrape_df = scrape(i)
        if scrape_df.empty:
            print(str(i) + " FAILED")
        else:
            scrape_df.set_index(pd.Index([i]), inplace=True)
            print(i)

        if i == min:
            final = scrape_df
        else:
            final = pd.concat([final, scrape_df])
    except:
        scrape_df = pd.DataFrame(columns=ej_columns)
        scrape_df = scrape_df.append(pd.Series(np.nan, index=scrape_df.columns), ignore_index=True)
        scrape_df.set_index(pd.Index([i]), inplace=True)
        if i == min:
            final = scrape_df
        else:
            final = pd.concat([final, scrape_df])


    #print(scrape_df)
    i = i + 1

uscities = uscities[min:max]

final_df = pd.DataFrame(final)
final_df = pd.concat([uscities, final], axis=1)

print(final_df)

final_df.to_csv('/scraping/PFAS_Stats_Final_Df_Rows_' + str(min) + '_to_' + str(max - 1) + '.csv')

In [ ]:
# # the code below replaces the primary EJ Screen data file from scraping

# import os
# import pandas as pd

# combine_files = os.listdir('/scraping')

# combine_files_df = []

# for combine_file in combine_files:
#     temp_df = pd.read_csv('/scraping/' + str(combine_file))
#     temp_df.set_index("Unnamed: 0", inplace=True)
#     temp_df.index.name = None
#     combine_files_df.append(temp_df)

# new_df = pd.concat(combine_files_df)
# new_df = new_df.sort_index()
# new_df.to_csv('/preprocessing/PFAS_EJ_Data.csv')

# Combine EJScreen and Industry Data

In [ ]:
import pandas as pd
ej_data = pd.read_csv('/preprocessing/PFAS_EJ_Data.csv')
ej_data.set_index("Unnamed: 0", inplace=True)
ej_data.index.name = None

industries_data = pd.read_csv('/preprocessing/PFAS_Industries_Data.csv')
industries_data.set_index("Unnamed: 0", inplace=True)
industries_data.index.name = None


final_pfas = pd.concat([industries_data, ej_data], axis=1)
final_pfas = final_pfas.loc[:,~final_pfas.columns.duplicated()]

final_pfas.insert(6, "Industry Presence", final_pfas["Industry Count"].apply(lambda x: 0 if x == 0 else 1))
final_pfas.to_csv("/preprocessing/PFAS_Data.csv")

# Scrape Testing Data

In [ ]:
#take city data for cLat/cLon here: https://simplemaps.com/data/us-cities
import math
import pandas as pd

earth_radius = 3960.0
degrees_to_radians = math.pi/180.0
radians_to_degrees = 180.0/math.pi
#asdasd

#Importing
industries = pd.read_csv("/content/drive/MyDrive/pfas_python_files/preprocessing/industries.csv", encoding = 'unicode_escape', engine ='python')
uscities = pd.read_csv("/content/drive/MyDrive/pfas_python_files/preprocessing/PFAS_Data.csv", encoding = 'unicode_escape', engine ='python')
testing = pd.read_csv("/content/drive/MyDrive/pfas_python_files/preprocessing/testing_sites.csv", encoding = 'unicode_escape', engine ='python')


#Cleaning Data
testing = testing[(testing['latitude'] != '-') & (testing['longitude'] != '-')]
testing['latitude'] = pd.to_numeric(testing['latitude'], errors='coerce')
testing['longitude'] = pd.to_numeric(testing['longitude'], errors='coerce')
testing = testing[['latitude', 'longitude']]


uscities['Latitude'] = pd.to_numeric(uscities['Latitude'], errors='coerce')
uscities['Longitude'] = pd.to_numeric(uscities['Longitude'], errors='coerce')


#Global Variables
upper_boundary_miles = 0
lower_boundary_miles = 0
right_boundary_miles = 0
left_boundary_miles = 0

grid_radius = 2.5


# 2.5 miles = around 0.04578754578 longitude
# 2.5 miles = around 0.03623188405 latitude


def calculate_testing_density_in_each_city():

    uscities['Test Sites Count'] = 0
    for index, row in uscities.iterrows():
        current_city_longitude = row['Longitude']
        current_city_latitude = row['Latitude']

        # Horizontal Boundaries
        right_boundary_longitude = current_city_longitude + 0.04578754578
        left_boundary_longitude = current_city_longitude - 0.04578754578

        # Vertical Boundaries
        upper_boundary_latitude = current_city_latitude + 0.03623188405
        lower_boundary_latitude = current_city_latitude - 0.03623188405

        testing_count = 0

        for ind_index, ind_row in testing.iterrows():
            current_testing_longitude = ind_row['longitude']
            current_testing_latitude = ind_row['latitude']

            # Check each testing point to see if it falls within the 2.5 mile radius of each city
            if (left_boundary_longitude < current_testing_longitude < right_boundary_longitude) and \
               (lower_boundary_latitude < current_testing_latitude < upper_boundary_latitude):
                testing_count += 1




        uscities.at[index, 'Testing Count'] = testing_count

        print("Index:", index)
        print("City:", row["city"])
        print("Testing Count :", testing_count)
        print("-------------------------------------")


calculate_testing_density_in_each_city()

columns_to_keep = ['city', 'state_name', 'Latitude', 'Longitude', 'Testing Count']

uscities = uscities[columns_to_keep]

uscities.to_csv('/preprocessing/PFAS_Testing_Data.csv')